In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
from bs4 import BeautifulSoup

In [2]:
def implied_volatility(target_value, call, s, k, t, r):
    limit = 100
    max_error = 1.0e-5
    sigma = 0.5

    for i in range(0, limit):
        price = bs(call, s, k, t, r, sigma)
        vega = vega(call, s, k, t, r, sigma)

        diff = target_value - price

        if (abs(diff) < max_error):
            return sigma
        sigma = sigma + diff/vega

    return sigma

def bs(call, s, k, t, r, v, q=0.0) -> float:
    d1 = (math.log(s / k) + (r + 0.5 * v**2) * t) / (v * math.sqrt(t))
    d2 = d1 - v * math.sqrt(t)
    if call:
        price = s * math.exp(-q * t) * norm.cdf(d1) \
                - k * math.exp(-r * t) * norm.cdf(d2)
    else:
        price = k * math.exp(-r * t) * norm.cdf(-d2) \
                - s * math.exp(-q * t) * norm.cdf(-d1)
    return price

def vega(s, k, t, r, v) -> float:
    d1 = (math.log(s / k) + (r + 0.5 * v**2) * t) / (v * math.sqrt(t))
    return s * math.sqrt(t) * norm.pdf(d1)

In [3]:
frame = pd.read_csv("csv/SI_D_ROPC.txt", sep="|", header=None, skiprows=1,
                    parse_dates=[3], 
                    names=["?", "stock", "stock type", "exercise date", "??", "???",
                           "series", "????", "strike price", "?????", "covered",
                           "uncovered", "total", "owner", "releaser", "??????",
                           "option type"])

frame[frame["stock"]=="PETR"].sort_values(by=["exercise date", "series"])
frame.filter(["stock", "exercise date", "series", "strike price"])
frame["call"] = frame["series"].str.get(4) < "L"

In [4]:
with open("html/Opções de compra | Valor Econômico.html") as f:
    html_string = f.read()
soup = BeautifulSoup(html_string, 'lxml')
table = soup.find_all('table')[0]

In [5]:
table

<table class="valor_tabela"><thead class="tabela-cabecalho-1"><tr class="row row-0 even first table-header"><th class="col col-0 first"><span></span></th><th class="col col-1"><span></span></th><th class="col col-2"><span></span></th><th class="col col-3"><span></span></th><th class="col col-4" colspan="3"><span>(Preços R$)</span></th><th class="col col-5"><span></span></th><th class="col col-6" colspan="1"><span>Oscil.</span></th><th class="col col-7" colspan="2"><span>Ofertas</span></th><th class="col col-8 last" colspan="2"><span>Neg. Realiz.</span></th></tr><tr class="row row-1 odd table-header"><th class="col col-0 first"><span>Cód.</span></th><th class="col col-1"><span>Emp./Ação</span></th><th class="col col-2"><span>Vcto.</span></th><th class="col col-3"><span>Preço</span></th><th class="col col-4"><span>Mín.</span></th><th class="col col-5"><span>Méd.</span></th><th class="col col-6"><span>Max.</span></th><th class="col col-7"><span>Fech.</span></th><th class="col col-8"><span

In [8]:
rows = table.find_all('tr', {'class': 'row'})[2:] # skip first 2 rows
rows

[<tr class="row row-2 even"><td class="col col-0 first">ABEVI228</td><td class="col col-1">ABEV        ON</td><td class="col col-2">set/19</td><td class="col col-3">22,57</td><td class="col col-4">-</td><td class="col col-5">-</td><td class="col col-6">-</td><td class="col col-7">-</td><td class="col col-8">-</td><td class="col col-9">-</td><td class="col col-10">0,75</td><td class="col col-11">-</td><td class="col col-12 last">-</td></tr>,
 <tr class="row row-3 odd"><td class="col col-0 first">ABEVH195</td><td class="col col-1">ABEV        ON</td><td class="col col-2">ago/19</td><td class="col col-3">19,23</td><td class="col col-4">-</td><td class="col col-5">-</td><td class="col col-6">-</td><td class="col col-7">-</td><td class="col col-8">-</td><td class="col col-9">-</td><td class="col col-10">0,30</td><td class="col col-11">-</td><td class="col col-12 last">-</td></tr>,
 <tr class="row row-4 even"><td class="col col-0 first">ABEVF160</td><td class="col col-1">ABEV        ON</td><

In [10]:
for i in rows[:10]:
    print(i.get_text())

ABEVI228ABEV        ONset/1922,57------0,75--
ABEVH195ABEV        ONago/1919,23------0,30--
ABEVF160ABEV        ONjun/1915,73-----0,05---
ABEVG165ABEV        ONjul/1916,57-----1,14---
ABEVF197ABEV        ONjun/1919,73------0,01--
ABEVG225ABEV        ONjul/1922,57------0,40--
ABEVF20ABEV        ONjun/1920,23------0,03--
ABEVF19ABEV        ONjun/1919,23------0,02--
ABEVF16ABEV        ONjun/1916,23-----0,85---
ABEVH182ABEV        ONago/1918,23-----0,100,70--


In [17]:
import re


regexp = r"^([A-Z]{5}\d+)([A-Z]{4})\s+[O|P|N]{2,4}([a-z]{3}/\d{2})(\d+.\d+)-*(\d+.\d+)"

for i in rows[:1]:
    x = re.search(regexp, i.get_text())
    print(x.groups())

# ^([A-Z]{5}\d+)([A-Z]{4})\s+[O|P|N]{2,4}([a-z]{3}/\d{2})(\d+.\d+)-*(\d+.\d+)

('ABEVI228', 'ABEV', 'set/19', '22,57', '0,75')


In [ ]:
options_frame = pd.DataFrame(columns=range(0,2), index = [0])

In [4]:
frame['implied volatility'] = frame.apply(lambda x: implied_volatility(x['value_1'],
                                                                       x['call'],
                                                                       29.85,
                                                                       x['strike price'],
                                                                       x['exercise date'],
                                                                       0.065), axis=1)

KeyError: ('value_1', 'occurred at index 0')